# Análise de Dados de Negócios com Pandas

## Contextualização

Vamos analisar os dados de 2019 de uma empresa prestadora de serviços fictícia. São três bases de dados:

- Cadastro de funcionários;
- Cadastro de clientes;
- Base de serviços prestados.

## O que queremos saber

Nossa análise descobrirá:

1) Custo total da folha salarial

2) Faturamento total da empresa

3) Percentual de funcionários que fecharam pelo menos um contrato

4) Total de contratos que cada área da empresa fechou

5) Número de funcionários em cada área da empresa

6) Tíquete médio mensal dos contratos

## Análise

### Importação das Bases

In [ ]:
# Importa a biblioteca pandas com o apelido pd
import pandas as pd

# Importa a biblioteca locale
import locale

# Define o local para que os números sejam formatados corretamente
locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

# Lê os arquivos especificados e os transforma em dataframes
df_clientes = pd.read_csv("CadastroClientes.csv", sep=";", decimal=",", encoding="utf-8")
df_funcionarios = pd.read_csv("CadastroFuncionarios.csv", sep=";", decimal=",", encoding="utf-8")
df_servicos_prestados = pd.read_excel("BaseServiçosPrestados.xlsx")

### Visualização dos Dataframes

Antes de começarmos a manipular essas bases de dados, vamos nos situar através da visualização das cinco primeiras linhas de cada dataframe.

#### Cadastro de Clientes

In [2]:
df_clientes.head()

,ID Cliente,Cliente,Valor Contrato Mensal
0,1,Teixeira Gonçalves,540
1,2,Souza Santos,1260
2,3,Emídio Alves,3195
3,4,Santos Costa,2520
4,5,Do Monteiro,3510


#### Cadastro de Funcionários

In [3]:
df_funcionarios.head()

,ID Funcionário,Estado Civil,Nome Completo,Salario Base,Impostos,Beneficios,VT,VR,Cargo,Area
0,1,C,Gabriel Mesquita,21910,10955.0,4382.0,242,719.04,Diretor,Operações
1,2,C,João Haddad,5404,2702.0,1080.8,154,574.56,Estagiário,Logística
2,3,C,Amanda Marques Ribeiro,16066,8033.0,3213.2,154,729.12,Estagiário,Administrativo
3,4,C,Guilherme Nunez,21305,10652.5,4261.0,220,524.16,Analista,Administrativo
4,5,C,Adelino Gomes,5098,2549.0,1019.6,176,725.76,Analista,Administrativo


As colunas __Estado Civil__ e __Cargo__ não têm qualquer relevância para a análise que vamos fazer. Portanto, vamos removê-las do dataframe.

In [4]:
df_funcionarios.drop(["Estado Civil", "Cargo"], axis=1, inplace=True)
df_funcionarios.head()

,ID Funcionário,Nome Completo,Salario Base,Impostos,Beneficios,VT,VR,Area
0,1,Gabriel Mesquita,21910,10955.0,4382.0,242,719.04,Operações
1,2,João Haddad,5404,2702.0,1080.8,154,574.56,Logística
2,3,Amanda Marques Ribeiro,16066,8033.0,3213.2,154,729.12,Administrativo
3,4,Guilherme Nunez,21305,10652.5,4261.0,220,524.16,Administrativo
4,5,Adelino Gomes,5098,2549.0,1019.6,176,725.76,Administrativo


#### Base de Serviços Prestados

In [5]:
df_servicos_prestados.head()

,Codigo do Servico,ID Funcionário,ID Cliente,Tempo Total de Contrato (Meses)
0,OS0001,67,1,14
1,OS0002,17,2,12
2,OS0003,116,4,14
3,OS0004,37,5,8
4,OS0005,130,6,8


### Respostas

#### Custo da Folha Salarial

In [6]:
# Cria uma nova coluna em que cada linha representa o custo total por funcionário
df_funcionarios["Custo Funcionario"] = df_funcionarios["Salario Base"] + df_funcionarios["Impostos"] + df_funcionarios["Beneficios"] + df_funcionarios["VT"] + df_funcionarios["VR"]
df_funcionarios.head()

,ID Funcionário,Nome Completo,Salario Base,Impostos,Beneficios,VT,VR,Area,Custo Funcionario
0,1,Gabriel Mesquita,21910,10955.0,4382.0,242,719.04,Operações,38208.04
1,2,João Haddad,5404,2702.0,1080.8,154,574.56,Logística,9915.36
2,3,Amanda Marques Ribeiro,16066,8033.0,3213.2,154,729.12,Administrativo,28195.32
3,4,Guilherme Nunez,21305,10652.5,4261.0,220,524.16,Administrativo,36962.66
4,5,Adelino Gomes,5098,2549.0,1019.6,176,725.76,Administrativo,9568.36


In [7]:
# Soma todos os valores da coluna "Custo Funcionario"
custo_folha_salarial = locale.currency(df_funcionarios["Custo Funcionario"].sum(), grouping=True)

# Exibe o resultado
print(f"Custo da folha salarial: {custo_folha_salarial}")

Custo da folha salarial: R$ 2.717.493,22


#### Faturamento da Empresa

Para calcular o faturamento da empresa, precisamos mesclar dois dataframes: o __df_clientes__ e o __df_servicos_prestados__. Enquanto o primeiro é uma tabela característica, o segundo é uma tabela fato. Precisamos nos atentar a isso antes de mesclar os dataframes, pois a tabela característica é a que deve ser mesclada à tabela fato. A mesclagem que vamos executar é semelhante a um INNER JOIN do SQL.

In [8]:
# Cria um novo dataframe, usando colunas dos dois dataframes
df_faturamento = df_servicos_prestados[["ID Cliente", "Tempo Total de Contrato (Meses)"]].merge(df_clientes[["ID Cliente", "Valor Contrato Mensal"]], on="ID Cliente")
df_faturamento.head()

,ID Cliente,Tempo Total de Contrato (Meses),Valor Contrato Mensal
0,1,14,540
1,2,12,1260
2,4,14,2520
3,5,8,3510
4,6,8,2340


In [9]:
# Cria uma nova coluna no dataframe com o faturamento de cada contrato
df_faturamento["Faturamento Contrato"] = df_faturamento["Tempo Total de Contrato (Meses)"] * df_faturamento["Valor Contrato Mensal"]
df_faturamento.head()

,ID Cliente,Tempo Total de Contrato (Meses),Valor Contrato Mensal,Faturamento Contrato
0,1,14,540,7560
1,2,12,1260,15120
2,4,14,2520,35280
3,5,8,3510,28080
4,6,8,2340,18720


In [10]:
# Soma todos os valores da coluna "Faturamento Contrato"
faturamento_total = locale.currency(df_faturamento["Faturamento Contrato"].sum(), grouping=True)

# Exibição do resultados
print(f"Faturamento da empresa: {faturamento_total}")

Faturamento da empresa: R$ 5.519.160,00


#### Percentual de Funcionários que Fecharam Contrato(s)

In [21]:
# Quantidade de funcionários cadastrados
qtde_funcionarios = df_funcionarios["ID Funcionário"].count()

# Quantidade de funcionários que fecharam ao menos um contrato
fecharam_contrato = len(df_servicos_prestados["ID Funcionário"].unique())

# Percentual de funcionário que fecharam ao menos um contrato
percentual_fecharam_contrato = fecharam_contrato / qtde_funcionarios

# Formatação do valor
percentual_fecharam_contrato = "{:.2%}".format(percentual_fecharam_contrato).replace(".", ",")

# Exibição de resultado
print(f"Percentual de funcionários que fecharam contratos: {percentual_fecharam_contrato}")

Percentual de funcionários que fecharam contratos: 86,84%


#### Número de Contratos por Área

In [22]:
# Cria um novo dataframe
df_areas = df_servicos_prestados[["ID Funcionário"]].merge(df_funcionarios[["ID Funcionário", "Area"]], on="ID Funcionário")
df_areas.head()

,ID Funcionário,Area
0,67,Financeiro
1,67,Financeiro
2,67,Financeiro
3,67,Financeiro
4,17,Logística


In [23]:
# Conta o número de contratos por cada área
df_areas["Area"].value_counts()

Administrativo    63
Operações         48
Comercial         44
Financeiro        42
Logística         40
Name: Area, dtype: int64

#### Número de Funcionários por Área

In [24]:
# Conta o número de funcionários por área
df_funcionarios["Area"].value_counts()

Administrativo    26
Comercial         26
Operações         23
Logística         21
Financeiro        18
Name: Area, dtype: int64

#### Faturamento Médio Mensal

In [25]:
# Calcula a média dos valores da coluna "Valor Contrato Mensal"
faturamento_medio_mensal = locale.currency(df_clientes["Valor Contrato Mensal"].mean(), grouping=True)

# Exibição do resultado
print(f"Faturamento médio mensal com contratos: {faturamento_medio_mensal}")

Faturamento médio mensal com contratos: R$ 2.502,56
